[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This is a template for DIY Covid Dashboard, where you can see the comparison of COVID occupied mechanical ventilator beds to new admissions. 

In [ ]:
pip install uk-covid19

In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install ipywidgets 

In [ ]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [ ]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={}

In [ ]:
with open("occupied_mvbeds.json") as json_file:
    data=json.load(json_file)
    print(data)

## Wrangle the data

Below is the logic to wrangle the raw data into a ```DataFrame``` that will be used for plotting.

In [ ]:
def wrangle_data(rawdata):
    startdate= '2020-01-31 00:00:00'
    enddate= '2022-09-11 00:00:00'
    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['hospital','mvbeds'])
    datalist=data['data']
    for entry in datalist: 
        date=entry['date']
        for column in ['hospital','mvbeds']:
            if pd.isna(df.loc[date, column]): 
                value= float(entry[column]) if entry[column]!= None else 0.0
                df.loc[date, column]=value
    df.fillna(0.0, inplace=True)
    return df

df=wrangle_data(jsondata)
df

In [ ]:
df = df[::-10]
df

## Download current data

Below is an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Note that the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In [ ]:
def access_api():
    filters = ['areaType=Overview']
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "mvbeds": "covidOccupiedMVBeds"}
    api = Cov19API(filters=filters, structure=structure)
    df =api.get_json()
    return df

def refresh_api(button):
    jsondata=access_api()
    global df
    df=wrangle_data(jsondata)
    refresh_graph()
    
apibutton=wdg.Button(
    description='Refresh',
    disabled=False,
    button_style='success',
    tooltip='Click to download current Public Health England data',
    icon='download')

apibutton.on_click(refresh_api)

display(apibutton)

In [ ]:
df.plot()

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

With below graph user can interact with the graph by clicking on "hospital" to view new admission through out the time period or "mvbeds" to check the number of COVID occupied mechanical ventilator beds through out time - or even compare both. The graph can be represented as linear model or log model. 

In [ ]:
occmvbeds=wdg.SelectMultiple(
    options=['hospital','mvbeds'],
    value=['hospital','mvbeds'],
    rows=2,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([occmvbeds, scale])

def occupied_mvbeds_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph=wdg.interactive_output(occupied_mvbeds_graph, {'gcols': occmvbeds, 'gscale': scale})

display(controls, graph)

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*